<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/classification_grouping_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import openai
import zipfile
import json
import os

In [2]:
!pip install openai

In [3]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [4]:
openai.api_key = ""

In [8]:
merged_ranking_path = "merged_ranking.txt"
ranking_res_zip = "/content/ranking_results.zip"

with zipfile.ZipFile(ranking_res_zip, 'r') as z, open(merged_ranking_path, 'w', encoding='utf-8') as out:
    for info in z.infolist():
        if info.filename.endswith('.txt'):
            out.write(f"===== {info.filename} =====\n")
            with z.open(info) as f:
                for raw in f:
                    out.write(raw.decode('utf-8', errors='replace'))
            out.write("\n\n")

print(f"Done — all .txt merged into `{merged_ranking_path}`")


Done — all .txt merged into `merged_ranking.txt`


In [7]:
# 1. Update these paths
obs_res_zip        = '/content/json_results.zip'
extract_dir     = '/content/extracted_jsons/json_results'
merged_json_path = 'merged_obs_extraction.json'

# 2. Extract JSON files
with zipfile.ZipFile(obs_res_zip, 'r') as zf:
    zf.extractall(extract_dir)

# 3. Merge into one dictionary, extending duplicates’ observables
merged = {}
for filename in os.listdir(extract_dir):
    # only process .json files
    if not filename.lower().endswith('.json'):
        continue

    full_path = os.path.join(extract_dir, filename)
    with open(full_path, 'r') as f:
        data = json.load(f)

    for md_name, doc in data.items():
        if md_name not in merged:
            merged[md_name] = doc
        else:
            # extend observables list from both
            merged[md_name]["observables"].extend(doc.get("observables", []))
            # you can merge other fields here if needed

# 4. Save the merged JSON
with open(merged_json_path, 'w') as f:
    json.dump(merged, f, indent=2)

In [ ]:
# Extract, lowercase, and dedupe classification values
unique_classifications = {
    obs['classification'].lower()
    for doc in merged.values()
    for obs in doc.get('observables', [])
    if 'classification' in obs
}

In [ ]:
gross_classifications = set([
"System",
"Module",
"Hardware",
"Device",
"PLC",
"POU",
"Identifier",
"Information",
"Credential",
"Data",
"Command",
"API",
"Code",
"Process",
"Configuration",
"Log",
"File",
"Payload",
"Software",
"Service",
"Firmware",
"Server",
"Communication",
"Protocol",
"Protocol Field",
"Connection Port",
"Vulnerability",
"Attack Technique"])

In [ ]:
prompt = f"""
You are a cybersecurity analyst expert. Your job is to group a list of observable “classification” labels under gross classifications from MITRE ATT&CK® Enterprise, or if not matched to any gross classification - create a new category of gross classification.

---
### Input
   - A set of unique observable classifications:
     {sorted(unique_classifications)}

   - A set of gross classifications from MITRE ATT&CK® Enterprise:
     {sorted(gross_classifications)}

---
### Task
   - For each unique observable classifications, choose exactly one gross classification it best fits.
   - If it does not clearly belong to any of the gross classification labels given, assign it to a new category of your own choosing, appending “– new” to that category name (e.g. “Data Management – new”).

---
### Output
   - Return **only** a single JSON object.
   - Each key must be either one of the 28 gross classification label names or one of your “– new” categories.
   - Each value must be an array of the unique observable classifications assigned to that key.

---
### Response format (return *only* this JSON)

```json
{
  "Code": [
    "script file"
  ],
  "Command": [
    "command string"
  ],
  "Identifier": [
    "ip address",
    "domain"
  ],
  "Data": [
    "file format",
    "file signature"
  ]
}

"""

In [ ]:
!pip install openai==0.28

In [ ]:
# 4. Call ChatGPT with model o3-mini-high
response = openai.ChatCompletion.create(
    model="o3-mini-2025-01-31",
    messages=[
        {"role": "system", "content": "You are a helpful cybersecurity expert."},
        {"role": "user", "content": prompt}
    ],
)

# 5. Print the JSON response
print(response.choices[0].message.content)